In [2]:
import pandas as pd
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file")
else:
    print("✅ API key loaded successfully")


✅ API key loaded successfully


In [4]:
from langchain_community.document_loaders import CSVLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
import sqlite3

### making the SQLlite db 

In [5]:
CSV_PATH = "data/10final_merged_realestate_data.csv"
DB_PATH = "properties_sql.db"

# Load CSV
df = pd.read_csv(CSV_PATH)

# Create SQLite DB
conn = sqlite3.connect(DB_PATH)
df.to_sql("properties", conn, if_exists="replace", index=False)

conn.close()
print("✅ CSV loaded into SQLite")

✅ CSV loaded into SQLite


In [5]:
CSV_PATH = "data/10final_merged_realestate_data.csv"
DB_PATH = "properties_sql.db"

# Load CSV
df = pd.read_csv(CSV_PATH)

### loading the data in the Faiss db and next making the dos created by csv loaders , have the meta data appended 

In [6]:
csv_path = "data/10final_merged_realestate_data.csv"  
loader = CSVLoader(file_path=csv_path)
documents = loader.load()

print(f"✅ Loaded {len(documents)} documents from CSV")


✅ Loaded 83 documents from CSV


In [7]:
print(documents[0])

page_content='unique_property_id: cmff8vfoq0013vxp7h3onmg46
id_x: cmf53kkzy000fvcu8tx8jwjmr
projectType: RESIDENTIAL
projectName: Ashwini
projectCategory: STANDALONE
slug: luxury-ashwini-ashoknagar-chembur-mumbai-675058
projectAge: 
projectSummary: _
possessionDate: 2025-09-28 00:00:00
id_y: cmf53kl01000nvcu8ibut7fka
landmark: Babys school
fullAddress: Mumbai chembur
pincode: 411017
propertyCategory: RESIDENTIAL
type: 1BHK
configurationId: cmf53kkzz000ivcu89r5399s4
bathrooms: 1
balcony: 1.0
furnishedType: UNFURNISHED
furnishingType: []
floorPlanImage: https://pub-d28896f69c604ec5aa743cb0397740d9.r2.dev/1757584023815-67012c27580e3e23.jpg
carpetArea: 123.0
price: 11111111
propertyImages: ["https://pub-d28896f69c604ec5aa743cb0397740d9.r2.dev/1756971672464-1e5179453b5df91d.jpg"]
lift: 0
ready_to_move: 0
context: Project Name: Ashwini and type :1BHK. Located at : Mumbai chembur , near luxury-ashwini-ashoknagar-chembur-mumbai-675058 , having landmark :Babys school Property Type: . Price: Rs1

In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [10]:
for i, doc in enumerate(documents):
    row = df.loc[i]

    # Update metadata
    doc.metadata.update({
        "unique_property_id": row.get("unique_property_id"),
        "price": row.get("price")
    })

In [11]:
documents[0]

Document(metadata={'source': 'data/10final_merged_realestate_data.csv', 'row': 0, 'unique_property_id': 'cmff8vfoq0013vxp7h3onmg46', 'price': np.int64(11111111)}, page_content='unique_property_id: cmff8vfoq0013vxp7h3onmg46\nid_x: cmf53kkzy000fvcu8tx8jwjmr\nprojectType: RESIDENTIAL\nprojectName: Ashwini\nprojectCategory: STANDALONE\nslug: luxury-ashwini-ashoknagar-chembur-mumbai-675058\nprojectAge: \nprojectSummary: _\npossessionDate: 2025-09-28 00:00:00\nid_y: cmf53kl01000nvcu8ibut7fka\nlandmark: Babys school\nfullAddress: Mumbai chembur\npincode: 411017\npropertyCategory: RESIDENTIAL\ntype: 1BHK\nconfigurationId: cmf53kkzz000ivcu89r5399s4\nbathrooms: 1\nbalcony: 1.0\nfurnishedType: UNFURNISHED\nfurnishingType: []\nfloorPlanImage: https://pub-d28896f69c604ec5aa743cb0397740d9.r2.dev/1757584023815-67012c27580e3e23.jpg\ncarpetArea: 123.0\nprice: 11111111\npropertyImages: ["https://pub-d28896f69c604ec5aa743cb0397740d9.r2.dev/1756971672464-1e5179453b5df91d.jpg"]\nlift: 0\nready_to_move: 0\n

In [12]:
import time
from tqdm import tqdm

# Process documents in smaller batches with rate limiting
batch_size = 10  # Adjust this based on your quota
delay_between_batches = 15  # seconds

print(f"Processing {len(documents)} documents in batches of {batch_size}...")

# Split documents into batches
all_embeddings = []
for i in tqdm(range(0, len(documents), batch_size)):
    batch = documents[i:i + batch_size]
    
    try:
        # Create FAISS index from batch
        if i == 0:
            # First batch - create new index
            vectordb = FAISS.from_documents(batch, embeddings)
        else:
            # Subsequent batches - merge with existing index
            batch_db = FAISS.from_documents(batch, embeddings)
            vectordb.merge_from(batch_db)
        
        print(f"✅ Processed batch {i//batch_size + 1}/{(len(documents)-1)//batch_size + 1}")
        
        # Rate limiting - wait between batches
        if i + batch_size < len(documents):
            time.sleep(delay_between_batches)
            
    except Exception as e:
        print(f"Error processing batch at index {i}: {e}")
        print(f"Try reducing batch_size or increasing delay_between_batches")
        raise

# Save the vector store
vectordb.save_local("faiss_realestate_index_with_metadata")
print("✅ FAISS vector store created and saved as 'faiss_realestate_index'")


Processing 83 documents in batches of 10...


  0%|          | 0/9 [00:00<?, ?it/s]

✅ Processed batch 1/9


 11%|█         | 1/9 [00:17<02:17, 17.21s/it]

✅ Processed batch 2/9


...

✅ Processed batch 7/9


 78%|███████▊  | 7/9 [01:56<00:33, 16.62s/it]

✅ Processed batch 8/9


100%|██████████| 9/9 [02:14<00:00, 14.90s/it]

✅ Processed batch 9/9
✅ FAISS vector store created and saved as 'faiss_realestate_index'


In [12]:
vectordb = FAISS.load_local(
    "faiss_realestate_index_with_metadata",
    embeddings,
    allow_dangerous_deserialization=True
)

print("✅ FAISS index loaded")

✅ FAISS index loaded


In [13]:
query = "1 bhk property in Mumbai near school"

results = vectordb.similarity_search(query, k=5)

for r in results:
    print("------")
    print(r.page_content[:20])
    print(r.metadata)


------
unique_property_id: 
{'source': 'data/10final_merged_realestate_data.csv', 'row': 18, 'unique_property_id': 'cmfc6pq1m0005vca07z1oktrg', 'price': np.int64(12000000)}
------
unique_property_id: 
{'source': 'data/10final_merged_realestate_data.csv', 'row': 0, 'unique_property_id': 'cmff8vfoq0013vxp7h3onmg46', 'price': np.int64(11111111)}
------
unique_property_id: 
{'source': 'data/10final_merged_realestate_data.csv', 'row': 5, 'unique_property_id': 'cmfawdrnr000cvc1897rpsu1b', 'price': np.int64(15000000)}
------
unique_property_id: 
{'source': 'data/10final_merged_realestate_data.csv', 'row': 15, 'unique_property_id': 'cmfaycwy9003cvc181yx3c5gf', 'price': np.int64(880000)}
------
unique_property_id: 
{'source': 'data/10final_merged_realestate_data.csv', 'row': 14, 'unique_property_id': 'cmfaycwy9003bvc189xf97r7w', 'price': np.int64(790000)}


In [17]:
from pydantic import BaseModel, Field
from typing import Optional, Literal

class SearchFilters(BaseModel):
    min_price: Optional[int] = Field(
        None, description="Minimum price in INR"
    )
    max_price: Optional[int] = Field(
        None, description="Maximum price in INR"
    )
    sort_by: Optional[Literal["price_asc", "price_desc"]] = Field(
        None, description="Sort order by price"
    )


In [18]:
from langchain_core.output_parsers import PydanticOutputParser

filter_parser = PydanticOutputParser(
    pydantic_object=SearchFilters
)

In [14]:
# Cell #VSC-1d9188c6 - CORRECTED
def sql_filter_by_price(max_price=None, min_price=None, sort_by=None):
    """
    Filter properties by price and return dict with prices for sorting.
    Returns: {property_id: price}
    """
    conn = sqlite3.connect("properties_sql.db")

    # FIXED: Changed to SELECT both columns
    query = "SELECT unique_property_id, price FROM properties"
    conditions = []
    params = []

    if min_price is not None:
        conditions.append("price >= ?")
        params.append(min_price)

    if max_price is not None:
        conditions.append("price <= ?")
        params.append(max_price)

    if conditions:
        query += " WHERE " + " AND ".join(conditions)

    if sort_by == "price_asc":
        query += " ORDER BY price ASC"
    elif sort_by == "price_desc":
        query += " ORDER BY price DESC"

    rows = conn.execute(query, params).fetchall()
    conn.close()

    # Now this works: row[0] = id, row[1] = price
    return {row[0]: row[1] for row in rows}

In [19]:
from langchain_core.prompts import ChatPromptTemplate

filter_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
You extract structured filters from real estate queries.

Rules:
- Extract price only if explicitly mentioned
- Convert lakh/crore to INR (1 crore = 10000000)
- Sorting only if user explicitly asks (low to high / high to low)
- If not mentioned, return null
- Do NOT guess values
"""
        ),
        ("human", "{query}"),
        ("human", "{format_instructions}")
    ]
)


In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0
)


filter_chain = (
    filter_prompt
    | llm
    | filter_parser
)


In [30]:
def search_properties(query: str):
    # 1️⃣ Extract filters
    filters = filter_chain.invoke({
        "query": query,
        "format_instructions": filter_parser.get_format_instructions()
    })

    # 2️⃣ SQL hard filter
    valid_id_price_map = sql_filter_by_price(
        min_price=filters.min_price,
        max_price=filters.max_price,
        sort_by=filters.sort_by
    )

    valid_ids = set(valid_id_price_map.keys())

    # 3️⃣ FAISS semantic recall
    faiss_results = vectordb.similarity_search(query, k=20)

    # 4️⃣ Intersection (HARD GUARANTEE)
    final_docs = [
        doc for doc in faiss_results
        if doc.metadata.get("unique_property_id") in valid_ids
    ][:5]

    return {
        "filters_used": filters,
        "results": final_docs
    }


In [31]:
from langchain_core.prompts import ChatPromptTemplate

explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a real estate assistant. "
         "Explain why the following properties match the query. "
         "Do not hallucinate."),
        ("human", "Query:\n{query}"),
        ("human", "Properties:\n{context}")
    ]
)

explain_chain = explain_prompt | llm


In [32]:
def explain_results(query, docs):
    if not docs:
        return "No matching properties found."

    context = "\n\n".join(
        f"ID: {d.metadata.get('unique_property_id')}\n{d.page_content}"
        for d in docs
    )

    return explain_chain.invoke({
        "query": query,
        "context": context
    }).content


In [33]:
response = search_properties("2 bhk under 1cr near metro having 2 lifts in high to low manner")

print("Filters:", response["filters_used"])

for d in response["results"]:
    print("----")
    print(d.metadata)
    print(d.page_content[:])

print("\nExplanation:")
print(explain_results("2 bhk under 1cr near metro", response["results"]))


Filters: min_price=None max_price=10000000 sort_by='price_desc'
----
{'source': 'data/10final_merged_realestate_data.csv', 'row': 33, 'unique_property_id': 'cmfcccifv004tvca0yjaewx04', 'price': np.int64(5770000)}
unique_property_id: cmfcccifv004tvca0yjaewx04
id_x: cmfcccifs004nvca0b7im3r5a
projectType: RESIDENTIAL
projectName: Balaji Kanha
projectCategory: STANDALONE
slug: balaji-kanha--ashoknagar-chembur-mumbai-678207
projectAge: 
projectSummary: _
possessionDate: 
id_y: cmfcccifw004vvca0mvtcgxos
landmark: Lodha Xperia Mall
fullAddress: 64C5+C63 Dombivli East Dombivli Maharashtra 421301
pincode: 421201
propertyCategory: RESIDENTIAL
type: 2BHK
configurationId: cmfcccifv004svca07g88pbp7
bathrooms: 2
balcony: 2.0
furnishedType: UNFURNISHED
furnishingType: []
floorPlanImage: https://pub-d28896f69c604ec5aa743cb0397740d9.r2.dev/1757409672776-9c48211b8fc08a50.jpg
carpetArea: 580.0
price: 5770000
propertyImages: ["https://pub-d28896f69c604ec5aa743cb0397740d9.r2.dev/1757409672776-55a3d0d5d7d3b

def hybrid_search(
    user_query,
    max_price=None,
    min_price=None,
    faiss_k=50,
    final_k=5
):
    """
    1. SQL applies HARD price constraints (if any)
    2. FAISS applies semantic ranking
    3. ID intersection ensures correctness
    """

    # --- Step 1: SQL filter (price optional)
    valid_ids = sql_filter_by_price(
        max_price=max_price,
        min_price=min_price
    )

    # --- Step 2: FAISS similarity search
    faiss_results = vectordb.similarity_search(
        user_query,
        k=faiss_k
    )

    # --- Step 3: ID intersection
    final_results = [
        doc for doc in faiss_results
        if doc.metadata.get("unique_property_id") in valid_ids
    ]

    return final_results[:final_k]
---------------------------------------
results = hybrid_search(
    user_query="1 bhk near school in Mumbai"
)

for r in results:
    print("------")
    print(r.page_content)
    print(r.metadata)

---------------------------------------

results = hybrid_search(
    user_query="1 bhk near school in Mumbai",
    max_price=12000000
)

for r in results:
    print("------")
    print(r.page_content)
    print(r.metadata)
    ------------------------------------------
results = hybrid_search(
    user_query="luxury apartment near metro",
    min_price=8000000,
    max_price=15000000
)

for r in results:
    print("------")
    print(r.page_content)
    print(r.metadata)
    

-----------------------------------
results = hybrid_search(
    user_query="cheap flat",
    max_price=2000000
)

if not results:
    print("❌ No properties found under the given budget. Try increasing the price range.")



In [21]:
from langchain_core.output_parsers import PydanticOutputParser

filter_parser = PydanticOutputParser(
    pydantic_object=SearchFilters
)


In [48]:
from typing import List, Optional
class PropertyMatch(BaseModel):
    id: str = Field(..., description="Unique property ID")
    projectName: Optional[str] = Field(None, description="Name of the real estate project")
    location: Optional[str] = Field(None, description="Project location or address")
    price: Optional[str] = Field(None, description="Price in INR")
    area: Optional[str] = Field(None, description="Total or built-up area")
    pincode: Optional[str] = Field(None, description="Project pincode")
    type: Optional[str] = Field(None, description="Property type: apartment, villa, plot, etc.")
    landmark: Optional[str] = Field(None, description="Nearby landmark")
    amenities: Optional[str] = Field(None, description="Mentioned amenities")

class RAGAnswer(BaseModel):
    matching_projects: List[PropertyMatch] = Field(default_factory=list)
    unmatched_points: List[str] = Field(default_factory=list)
    explanation: str = Field(..., description="Reasoning")


In [23]:
from langchain_core.prompts import ChatPromptTemplate

filter_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are a strict information extraction system.
Extract ONLY the fields defined in the schema.
If a value is not explicitly mentioned, return null.

Rules:
- Convert lakh/crore to INR numbers
- Do NOT guess values
- Do NOT add extra fields
- Sorting only if explicitly requested
"""
        ),
        ("human", "{query}"),
        ("human", "{format_instructions}")
    ]
)


In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0
)

filter_chain = (
    filter_prompt | llm | filter_parser
)


In [33]:
query = "List projects with apartments near subhash nagar with lift under 1cr"

filters = filter_chain.invoke({
    "query": query,
    "format_instructions": filter_parser.get_format_instructions()
})

min_price = filters.min_price
max_price = filters.max_price
sort_by = filters.sort_by



In [58]:
# Cell #VSC-9b5683f0 - UPDATED (should work now)
def hybrid_search(
    user_query,
    max_price=None,
    min_price=None,
    sort_by=None,
    faiss_k=50,
    final_k=5
):
    """
    Hybrid search combining FAISS semantic search + SQL price filtering.
    """

    # --- Step 1: SQL filter - NOW returns {id: price} dict
    valid_properties = sql_filter_by_price(
        min_price=min_price,
        max_price=max_price,
        sort_by=sort_by
    )

    # --- Step 2: FAISS similarity search
    faiss_results = vectordb.similarity_search(
        user_query,
        k=faiss_k
    )

    # --- Step 3: ID intersection - check if ID is in dict keys
    final_results = [
        doc for doc in faiss_results
        if doc.metadata.get("unique_property_id") in valid_properties
    ]    
        
    # --- Step 4: Sort results using price from dict
    if sort_by == "price_asc":
        final_results = sorted(
            final_results,
            key=lambda doc: valid_properties.get(doc.metadata.get("unique_property_id"), float('inf'))
        )
    elif sort_by == "price_desc":
        final_results = sorted(
            final_results,
            key=lambda doc: valid_properties.get(doc.metadata.get("unique_property_id"), 0),
            reverse=True
        )
    
    return final_results[:final_k]

In [52]:
rag_parser = PydanticOutputParser(pydantic_object=RAGAnswer)

rag_prompt = ChatPromptTemplate.from_template("""
You are a Real Estate Expert Assistant.

Retrieved Property Data:
{context}

User Query:
{question}

Instructions:
1. Extract matching properties from the context above
2. Map each property to these fields: id, projectName, location, price, area, pincode, type, landmark, amenities
3. List any unmatched query conditions under unmatched_points
4. Never hallucinate or assume data not in the context
5. If no properties match, leave matching_projects empty

{format_instructions}
""")

rag_chain = rag_prompt | llm | rag_parser


In [53]:
query = "List projects with apartments near subhash nagar with lift under 1cr, sort by cheapest"

# Extract filters from query using LLM
filters = filter_chain.invoke({
    "query": query,
    "format_instructions": filter_parser.get_format_instructions()
})

min_price = filters.min_price
max_price = filters.max_price
sort_by = filters.sort_by

print(f"📋 Extracted Filters:")
print(f"  Min Price: ₹{min_price:,}" if min_price else "  Min Price: None")
print(f"  Max Price: ₹{max_price:,}" if max_price else "  Max Price: None")
print(f"  Sort By: {sort_by if sort_by else 'None'}\n")


📋 Extracted Filters:
  Min Price: None
  Max Price: ₹10,000,000
  Sort By: price_asc



In [61]:
hybrid_results = hybrid_search(
    user_query=query,
    max_price=max_price,
    min_price=min_price,
    sort_by=sort_by,
    faiss_k=50,
    final_k=5
)

In [62]:
print(f"🔍 Hybrid Search Results: {len(hybrid_results)} properties found")
if sort_by:
    print(f"📊 Sorted by: {sort_by}\n")
else:
    print()


🔍 Hybrid Search Results: 5 properties found
📊 Sorted by: price_asc



In [ ]:
context = "\n\n---\n\n".join([
    f"""Property ID: {doc.metadata.get("unique_property_id", "N/A")}
Price: ₹{doc.metadata.get("price", "N/A"):,}
Content:
{doc.page_content}"""
    for doc in hybrid_results
])

# --- Invoke RAG chain
input_data = {
    "context": context,
    "question": query,
    "format_instructions": rag_parser.get_format_instructions()
}
query='Which projects have  apartments near Babys school?'
rag_response = rag_chain.invoke(input_data)
print(rag_response)

matching_projects=[] unmatched_points=['apartments near subhash nagar', 'with lift'] explanation="No properties were found that match all the specified criteria. Specifically, no properties were found near 'subhash nagar' or with a 'lift' amenity."


In [64]:
print(rag_response)

matching_projects=[] unmatched_points=['apartments near subhash nagar', 'with lift'] explanation="No properties were found that match all the specified criteria. Specifically, no properties were found near 'subhash nagar' or with a 'lift' amenity."
